# Filtering

这是陈硕写的《数字信号处理入门实验》的第二个实验，介绍数字滤波器的设计与使用。
最新版网址： http://github.com/chenshuo/notes

如果想要执行交互式的内容，可以用 Colab 打开：
https://colab.research.google.com/github/chenshuo/notes/blob/master/dsp_labs/2-filtering.ipynb

本章内容的视频讲解在
* [FIR 低通滤波](https://www.youtube.com/watch?v=NVv4xhbmn_Y)  国内：https://www.bilibili.com/video/BV1ja411S7dM
* [IIR 滤波、陷波](https://www.youtube.com/watch?v=Bxf29Vc45Sg)  国内：https://www.bilibili.com/video/BV1pe4y1X7J5

照例先引入 NumPy、SciPy.signal、Matplotlib、LibROSA 等必要的库，再定义两个常用的绘图函数。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

import librosa as rosa
import librosa.display
from IPython.display import Audio

from ipywidgets import interact

np.set_printoptions(suppress=True)

In [ ]:
def draw_pzmap(dlti):
  plt.plot(np.real(dlti.zeros), np.imag(dlti.zeros), 'o', mfc='none')
  plt.plot(np.real(dlti.poles), np.imag(dlti.poles), 'x')
  w = np.linspace(0, 2*np.pi)
  plt.plot(np.cos(w), np.sin(w), 'y--')
  limits = plt.axis("equal")
  plt.grid()

def draw_resp_stem(x, y):
  plt.stem(x, use_line_collection=True)
  (markerline, _, _) = plt.stem(y, linefmt='r', markerfmt='ro', use_line_collection=True)
  markerline.set_markerfacecolor('none')

## FIR

$H(\omega) = H(z)|_{z=e^{j\omega}}$, $0 \le \omega \le \pi$

$H(z) =  \frac{b_0 + b_1z^{-1}+b_2z^{-2}+\cdots+b_Nz^{-N}}{1+a_1z^{-1}+a_2z^{-2}+\cdots+a_Mz^{-M}}$

For FIR, $a_i = 0,\ i > 0$

$H(z) = b_0 + b_1z^{-1}+b_2z^{-2}+\cdots+b_Nz^{-N}$

$H(w) = H(z)|_{z=e^{jw}} = b_0 + b_1e^{-1jw}+b_2e^{-2jw}+\cdots+b_Ne^{-jNw}$

$H(w) = \sum_{k=0}^N b_k e^{-jkw}$, this is DTFT.

Choose $b_k$ to minimize difference between $H(w)$ and the desired frequency response. 

Parks-McClellan algorithm 1973.

https://eeweb.engineering.nyu.edu/iselesni/EL713/remez/remez.pdf

In [ ]:
x, sr = rosa.load(rosa.example('brahms'), sr=None)
print("Sample rate: %d, original length %.2f sec" % (sr, len(x) / sr))

x = x[0:int(sr*13.6)]
plt.plot(np.arange(len(x))/sr, x)
Audio(x, rate=sr, normalize=False)

In [ ]:
f1 = rosa.stft(x)
rosa.display.specshow(rosa.amplitude_to_db(np.abs(f1)), sr=sr, y_axis='hz', x_axis='s')

固定电话的带宽是 200 Hz ~ 3.4 kHz，这里用低通模拟。

三个相互制约的因素：
* tap 数
* 阻带衰减 (dB)
* 过渡带宽度 (Hz)

In [ ]:
numtaps = 128
h = signal.remez(numtaps, bands=[0, 3400, 4000, 0.5*sr], desired=[1, 0], fs=sr)

dlti = signal.dlti(h, [1])
w, mag, phase = dlti.bode()
plt.plot(w/(2*np.pi)*sr, mag)
plt.ylabel('dB'); plt.xlabel('Hz')

In [ ]:
plt.plot(h)

In [ ]:
y = signal.oaconvolve(x, h, mode='full')  # 'full', 'valid', 'same'
print('%.3f' % (len(y)/sr))
plt.plot(y)
Audio(y, rate=sr, normalize=False)

In [ ]:
f2 = rosa.stft(y)
rosa.display.specshow(rosa.power_to_db(np.abs(f2)**2), sr=sr, y_axis='hz', x_axis='s')

In [ ]:
draw_pzmap(dlti)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
rosa.display.specshow(rosa.power_to_db(np.abs(f1)**2), sr=sr, y_axis='hz', x_axis='s')
plt.subplot(122)
rosa.display.specshow(rosa.power_to_db(np.abs(f2)**2), sr=sr, y_axis='hz', x_axis='s')

可见 4kHz 以上的高频被滤得干干净净。

当然，这只是原理验证，离实时的定点 DSP 实现还有相当的距离。

## DC blocker

Julius O. Smith, Introduction to Digital Filters with Audio Applications, http://ccrma.stanford.edu/~jos/filters/

Appendix B.3 https://ccrma.stanford.edu/~jos/filters/DC_Blocker.html

$y[n] = x[n]-x[n-1]+Ry[n-1]$

$H(z)=\dfrac{1-z^{-1}}{1-Rz^{-1}}$

It's a differentiator with exponential moving average.

In [ ]:
R = 0.9
b = [1, -1]
a = [1, -R]
dlti = signal.dlti(b, a)
zeros = dlti.zeros
print('Zeros:', zeros)
print('Poles:', dlti.poles)
draw_pzmap(dlti)

In [ ]:
w, mag = dlti.freqresp()
plt.plot(w/np.pi, np.abs(mag))

In [ ]:
dc = np.concatenate([np.zeros(100), np.ones(150), -1 * np.ones(150)])
y = signal.lfilter(b, a, dc)
plt.plot(dc)
plt.plot(y)

In [ ]:
t = np.linspace(0, 1, 400) * 2 *np.pi * 15
x = np.sin(t) + dc

y = signal.lfilter(b, a, x)
plt.plot(x)
plt.plot(y)

In [ ]:
t = np.linspace(0, 1, 400) * 2 *np.pi
x = np.sin(t * 15)
triangle = signal.sawtooth(t, 0.5)
x = x + triangle

y = signal.lfilter(b, a, x)
plt.plot(x)
plt.plot(y)

In [ ]:
@interact(R=(0, 0.99, 0.01))
def blocker(R=0.88):
  b = [1, -1]
  a = [1, -R]
  dlti = signal.dlti(b, a)
  plt.figure(figsize=(15,5))
  plt.subplot(121)
  draw_pzmap(dlti)
  w, mag = dlti.freqresp()
  plt.subplot(122)
  plt.plot(w/np.pi, np.abs(mag))

## IIR Notch

https://www.mathworks.com/help/signal/ug/remove-the-60-hz-hum-from-a-signal.html

Sophocles J. Orfanidis. _Introduction to Signal Processing, 1996._ 

§11.3 Second-Order Peaking and Notching Filters

https://www.ece.rutgers.edu/~orfanidi/intro2sp/

In [ ]:
fs=1000
b, a = signal.iirnotch(60, Q=30, fs=fs)
d = signal.dlti(b, a)
print('b=', np.round(b, 5))
print('a=', np.round(a, 5))

$y[n] = 0.99376\,x[n] - 1.84794\,x[n-1] + 0.99376\,x[n-2] + 1.84794\,y[n-1] - 0.98751\,y[n-2]$

In [ ]:
w, amp = d.freqresp(n=1000)
w=w[0:300]
amp=amp[0:300]
plt.plot(w/(2*np.pi)*fs, np.abs(amp))

In [ ]:
w, mag, phase = d.bode(n=200)
plt.plot(w/np.pi, mag)
plt.figure()
plt.plot(w/np.pi, phase)

In [ ]:
t, y = d.impulse(n=500)
plt.plot(t, y[0])

In [ ]:
t, y = d.step(n=500)
plt.plot(t, y[0])

In [ ]:
print('zeros %.5f %.3fHz' % (np.abs(d.zeros[0]), np.angle(d.zeros[0]) * fs / (2*np.pi)))
print('poles %.5f %.3fHz' % (np.abs(d.poles[0]), np.angle(d.poles[0]) * fs / (2*np.pi)))
draw_pzmap(d)

### EEG

In [ ]:
eeg = np.loadtxt('data/eeg.txt').T
fs = 500
x = eeg[0]
t = np.arange(len(x)) / fs
_ = plt.plot(t, x)

In [ ]:
fft = np.fft.rfft(x) / len(x)
freq = np.arange(len(fft)) / len(fft) * fs / 2
plt.plot(freq, np.abs(fft))

In [ ]:
plt.plot(freq, np.abs(fft))
plt.xlim(40, 60)
plt.ylim(0, 0.5)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(freq, np.abs(fft))
plt.xlim(140, 160)
plt.ylim(0, 1.6)
plt.subplot(122)
plt.semilogy(freq, np.abs(fft))
plt.xlim(140, 160)

In [ ]:
b, a = signal.iirnotch(150, Q=50, fs=fs)
d = signal.dlti(b, a)
print('b=', np.round(b, 5))
print('a=', np.round(a, 5))

In [ ]:
y = signal.lfilter(b, a, x)
ffty = np.fft.rfft(y) / len(y)
freq = np.arange(len(ffty)) / len(ffty) * fs / 2
plt.plot(freq, np.abs(ffty))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(freq, np.abs(ffty))
plt.xlim(140, 160)
plt.ylim(0, 1.6)
plt.subplot(122)
plt.semilogy(freq, np.abs(ffty))
plt.xlim(140, 160)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(freq, np.abs(fft))
plt.subplot(122)
plt.plot(freq, np.abs(ffty))

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(freq, np.abs(fft)-np.abs(ffty))
plt.subplot(122)
plt.plot(freq, np.abs(fft)-np.abs(ffty))
plt.xlim(140, 160)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t, x)
plt.subplot(122)
plt.plot(t, y)

In [ ]:
diff = y-x
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t, diff)
plt.subplot(122)
plt.plot(t, diff)
plt.xlim(1, 1.1)

In [ ]:
fftd = np.fft.rfft(diff) / len(diff)
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(freq, np.abs(fftd))
plt.subplot(122)
plt.plot(freq, np.abs(fftd))
plt.xlim(140, 160)

FFT(y - x) = FFT(y) - FFT(x)

**练习**： Remove 15.625 kHz noise from CD recordings?


**THD**

https://www.youtube.com/watch?v=Zvireu2SGZM&t=831s